In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import pickle

In [28]:
import tensorflow as tf
# tensor flow functions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, InputLayer, Dropout,GaussianNoise,RandomContrast
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [4]:
# define path to image folders
img_folders_path = '256_ObjectCategories/sim_images/'

In [5]:
# from tensorflow.python.ops.numpy_ops import np_config
# np_config.enable_numpy_behavior()

In [7]:
# load data
datagen = ImageDataGenerator(rescale=1./255, # converts pixels in range 0,255 to between 0 and 1
                             # this will make every image contribute more evenly to the total loss
                            validation_split=0.2)

# if want to implement gray scale:
def to_grayscale(image):
    image = tf.image.rgb_to_grayscale(image)
    return image

# then use preprocessing_function=to_grayscale in ImageDataGenerator
traingen_g = ImageDataGenerator(rescale=1./255,
                                validation_split = 0.2,
                                preprocessing_function=to_grayscale)

# if split into train and test dirs, could apply brightness range to
# only test set using "brightness_range" option in ImageDataGenerator
traingen_b = ImageDataGenerator(rescale=1./255,
                                validation_split = 0.2,
                                brightness_range = [0.5,1.5])

train_generator = datagen.flow_from_directory(
    img_folders_path,
    target_size=(150, 150),
    batch_size=32,
    shuffle = True,
    class_mode='categorical',
    subset='training'
)

# in case I want to experiment with brightness as an augmentation
# train_generator_b = traingen_b.flow_from_directory(
#     img_folders_path,
#     target_size=(150, 150),
#     batch_size=32,
#     shuffle = True,
#     class_mode='categorical',
#     subset='training'
# )

# or with gray scale as augmentation
# train_generator_g = traingen_g.flow_from_directory(
#     img_folders_path,
#     target_size=(150, 150),
#     batch_size=32,
#     shuffle = True,
#     class_mode='categorical',
#     subset='training'
# )


# will use this validation set as a test set, as val metrics can be used for early stopping and ability for tuning,
# but do not affect training (according to search I made)
test_generator = datagen.flow_from_directory(
    img_folders_path,
    target_size=(150, 150),
    batch_size=32,
    shuffle = True,
    class_mode='categorical',
    subset='validation',
)

Found 4232 images belonging to 46 classes.
Found 1038 images belonging to 46 classes.


In [9]:
# define important parameters
num_classes = len(np.unique(train_generator.classes))
noise_sd = 0.1

In [10]:
# create early stopping
es = EarlyStopping(monitor='loss',patience=3,mode='min')
# could also add "start_from_epoch" specification in this es object
# in .fit() add the following to implement early stopping:
# callbacks=[es]

In [12]:
model_ffn = Sequential([
    RandomContrast(0, input_shape=(150, 150, 3)),
    GaussianNoise(0),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.4),
    # GaussianNoise(noise_sd),
    Dense(256, activation='relu'),
    Dropout(0.2),
    # GaussianNoise(noise_sd),
    Dense(128, activation='relu'),
    Dropout(0.1),
    # 5 classes: so get 5 output nuerons with softmax activiation function (to give probability of each class)
    Dense(num_classes, activation='softmax')
])
# there may also be some overfitting, so using some dropout could be helpful

model_ffn.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

model_ffn.fit(train_generator, epochs=10,
            #   steps_per_epoch=10,
                callbacks=[es])
model_ffn.summary()

Epoch 1/10
 20/133 [===>..........................] - ETA: 1:24 - loss: 4.9638 - accuracy: 0.0219

KeyboardInterrupt: 

In [52]:
# model_ffn.history.history

{'loss': [5.510605812072754, 4.4738874435424805],
 'accuracy': [0.01875000074505806, 0.03750000149011612]}

In [86]:
model_cnn = Sequential([
    # syntax: Conv2D(num_filters, kernel_size (shape1,shape2), activation, input_shape for first layer)
    # RandomContrast(0.2, input_shape=(150, 150, 3)),
    # GaussianNoise(noise_sd),
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    # GaussianNoise(noise_sd),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    # GaussianNoise(noise_sd),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    # after convolutional layers, flatten the output, then use 1-2(/3?) dense layers
    Flatten(),
    # GaussianNoise(noise_sd),
    Dense(512, activation='relu'),
    Dropout(0.4),
    # GaussianNoise(noise_sd),
    Dense(256, activation='tanh'),
    Dropout(0.2),
    # GaussianNoise(noise_sd),
    # Dense(128, activation='relu'),
    # Dropout(0.1),
    Dense(num_classes, activation='softmax')
])

model_cnn.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

model_cnn.fit(train_generator, epochs=10,
              callbacks = [es])
# default is to take len(train_generator steps per epoch)
model_cnn.summary()

Epoch 1/10
  4/133 [..............................] - ETA: 1:09 - loss: 3.8790 - accuracy: 0.0312  

KeyboardInterrupt: 

In [88]:
# save and restore models
model_ffn.save("ffn_model")
model_ffn_nc.save("ffn_model_nc")
model_ffn_c.save("ffn_model_c")
model_ffn_n.save("ffn_model_n")

model_cnn.save("cnn_model")
model_cnn_nc.save("cnn_model_nc")
model_cnn_c.save("cnn_model_c")
model_cnn_n.save("cnn_model_n")


INFO:tensorflow:Assets written to: ffn_model\assets
INFO:tensorflow:Assets written to: ffn_model_nc\assets


KeyboardInterrupt: 

In [6]:

# # reload the model
model_ffn_reload = tf.keras.models.load_model('ffn_model')
model_ffn_nc_reload = tf.keras.models.load_model('ffn_model_nc')
# model_cnn_reload = tf.keras.models.load_model('cnn_model')
# saver = tf.train.Saver(max_to_keep=1)
# with tf.Session() as sess:
#     # train your model, then:
#     savePath = saver.save(sess, 'my_ffn.ckpt')
# # To restore:

# with tf.Session() as sess:
#     saver = tf.train.import_meta_graph('my_ffn.ckpt.meta')
#     saver.restore(sess, pathModel + 'someDir/my_model.ckpt')
#     # access a variable from the saved Graph, and so on:
#     someVar = sess.run('varName:0')

In [44]:
results = model_ffn_reload.predict(test_generator)

33/33 [==============================] - 14s 436ms/step


In [ ]:

# # Reset 
# val_data_gen.reset()

# # Evaluate on Validation data
# scores = model.evaluate(val_data_gen)
# print("%s%s: %.2f%%" % ("evaluate ",model.metrics_names[1], scores[1]*100))

In [15]:
# test out functionality to
n_sim = 2
test_accs = [np.nan]*n_sim

In [16]:
# evaluate the model
# _, train_acc = model_ffn.evaluate(train_generator, verbose=0)
_, test_accs[j] = model_ffn_reload.evaluate(test_generator)
_, test_accs[1] = model_ffn_nc_reload.evaluate(test_generator)
print(test_accs)

33/33 [==============================] - 10s 299ms/step - loss: 4.5930 - accuracy: 0.0173
[0.023121386766433716, 0.017341040074825287]


In [69]:
pred_digits = np.argmax(results,axis=1)
test_labels = test_generator.classes
len(test_labels)
len(pred_digits)

1038

In [51]:
# combine model fits and accuracy assessments into one function
def network_sim(train_generator,test_generator,n_epochs,noise_sd=0,contrast_factor=0):
    """Fit Feed Forward and Convolutional Neural Networks with 2 factors: Random Noise and Random Contrast
    parameters:
        train_generator
        test_generator
        noise_sd: float [0.0,1.0], default = 0
            sd of Gaussian(0,sd) random noise to be added during training
        contrast_factor: float [0.0,1.0], defauly = 0
            factor to scale random contrast adjustment of images; 
            factor applied using following formula: 
            layer computes the mean of the image pixels in the channel and then adjusts each component x
            of each pixel to (x - mean) * contrast_factor + mean
    returns: *dictionary* {FFN_accuracy,CNN_accuracy}
    """
    # get number of classes in training data
    num_classes = len(np.unique(train_generator.classes))
    # define early stopping criteria
    es = EarlyStopping(monitor='loss',patience=3,mode='min')

    # feed forward nn fit
    model_ffn_nc = Sequential([
        RandomContrast(factor=contrast_factor, input_shape=(150, 150, 3)),
        GaussianNoise(noise_sd),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.2),
        # GaussianNoise(noise_sd),
        Dense(256, activation='relu'),
        Dropout(0.1),
        # GaussianNoise(noise_sd),
        Dense(128, activation='relu'),
        Dropout(0.1),
        # 5 classes: so get 5 output nuerons with softmax activiation function (to give probability of each class)
        Dense(num_classes, activation='softmax')
    ])
    # there may also be some overfitting, so using some dropout could be helpful

    model_ffn_nc.compile(optimizer=Adam(learning_rate=0.0001),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
    print(f'Fitting feed forward network with Gaussian(0,{noise_sd}) noise and {contrast_factor} random contrast')
    model_ffn_nc.fit(train_generator, epochs=n_epochs,
                #   steps_per_epoch=10,
                    callbacks=[es])
    

    # cnn fit
    model_cnn_nc = Sequential([
        # syntax: Conv2D(num_filters, kernel_size (shape1,shape2), activation, input_shape for first layer)
        # recall: adding dropout can be helpful to remedy overfitting
        RandomContrast(factor=contrast_factor, input_shape=(150, 150, 3)),
        GaussianNoise(noise_sd),
        Conv2D(32, (3, 3), activation='relu'), #input_shape=(150, 150, 3)),
        MaxPooling2D(2, 2),
        # Dropout(0.1),
        # GaussianNoise(noise_sd),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        # Dropout(0.1),
        # GaussianNoise(noise_sd),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        # Dropout(0.1),
        Conv2D(256, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        # Dropout(0.1),
        # after convolutional layers, flatten the output, then use 1-2(/3?) dense layers
        Flatten(),
        # GaussianNoise(noise_sd),
        Dense(512, activation='relu'),
        Dropout(0.2),
        # GaussianNoise(noise_sd),
        Dense(256, activation='tanh'),
        Dropout(0.2),
        # GaussianNoise(noise_sd),
        # Dense(128, activation='relu'),
        # Dropout(0.1),
        Dense(num_classes, activation='softmax')
    ])

    model_cnn_nc.compile(optimizer=Adam(learning_rate=0.0001),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

    print(f'Fitting CNN with Gaussian(0,{noise_sd}) noise and {contrast_factor} random contrast')
    model_cnn_nc.fit(train_generator, epochs=n_epochs,
                    callbacks = [es])
    # default is to take len(train_generator steps per epoch)

    # accuracy evaluations
    _, test_acc_ffn = model_ffn_nc.evaluate(test_generator)
    _, test_acc_cnn = model_cnn_nc.evaluate(test_generator)

    return {"FFN_accuracy": test_acc_ffn, "CNN_accuracy": test_acc_cnn}

In [34]:
# test network_sim
baseline_results = network_sim(train_generator,test_generator,n_epochs=1)
nc_results = network_sim(train_generator,test_generator,n_epochs=1,noise_sd=0.2,contrast_factor=0.1)
n_results = network_sim(train_generator,test_generator,n_epochs=1,noise_sd=0.2,contrast_factor=0)
c_results = network_sim(train_generator,test_generator,n_epochs=1,noise_sd=0,contrast_factor=0.1)


Fitting feed forward network with Gaussian(0,0) and 0 random contrast
133/133 [==============================] - 51s 369ms/step - loss: 4.0248 - accuracy: 0.0397
Fitting CNN with Gaussian(0,0) and 0 random contrast
33/33 [==============================] - 13s 372ms/step - loss: 3.6911 - accuracy: 0.1050
Fitting feed forward network with Gaussian(0,0.2) and 0.1 random contrast
133/133 [==============================] - 52s 363ms/step - loss: 4.0177 - accuracy: 0.0378
Fitting CNN with Gaussian(0,0.2) and 0.1 random contrast
33/33 [==============================] - 12s 353ms/step - loss: 3.7675 - accuracy: 0.0684
Fitting feed forward network with Gaussian(0,0.2) and 0 random contrast
133/133 [==============================] - 45s 321ms/step - loss: 3.9837 - accuracy: 0.0352
Fitting CNN with Gaussian(0,0.2) and 0 random contrast
33/33 [==============================] - 12s 350ms/step - loss: 3.7489 - accuracy: 0.0694
Fitting feed forward network with Gaussian(0,0) and 0.1 random contrast
1

In [35]:
# test list structure before aplying in loop
n_sim = 3
ffn_base_accs = [np.nan]*n_sim
ffn_nc_accs = [np.nan]*n_sim
ffn_n_accs = [np.nan]*n_sim
ffn_c_accs = [np.nan]*n_sim
cnn_base_accs = [np.nan]*n_sim
cnn_nc_accs = [np.nan]*n_sim
cnn_n_accs = [np.nan]*n_sim
cnn_c_accs = [np.nan]*n_sim

ffn_base_accs[0] = baseline_results["FFN_accuracy"]
ffn_nc_accs[0] = nc_results["FFN_accuracy"]
ffn_n_accs[0] = n_results["FFN_accuracy"]
ffn_c_accs[0] = c_results["FFN_accuracy"]

cnn_base_accs[0] = baseline_results["CNN_accuracy"]
cnn_nc_accs[0] = nc_results["CNN_accuracy"]
cnn_n_accs[0] = n_results["CNN_accuracy"]
cnn_c_accs[0] = c_results["CNN_accuracy"]

In [52]:
# repeat process for simulation study
# then can also decide if we want to up noise/contrast or change filter

n_sims = 20
# initialize result lists
ffn_base_accs = [np.nan]*n_sim
ffn_nc_accs = [np.nan]*n_sim
ffn_n_accs = [np.nan]*n_sim
ffn_c_accs = [np.nan]*n_sim
cnn_base_accs = [np.nan]*n_sim
cnn_nc_accs = [np.nan]*n_sim
cnn_n_accs = [np.nan]*n_sim
cnn_c_accs = [np.nan]*n_sim
for j in range(n_sims):
    print(f"run {j+1}")
    # train networks
    baseline_results = network_sim(train_generator,test_generator,n_epochs=1)
    nc_results = network_sim(train_generator,test_generator,n_epochs=1,noise_sd=0.2,contrast_factor=0.1)
    n_results = network_sim(train_generator,test_generator,n_epochs=1,noise_sd=0.2,contrast_factor=0)
    c_results = network_sim(train_generator,test_generator,n_epochs=1,noise_sd=0,contrast_factor=0.1)

    # export results
    ffn_base_accs[j] = baseline_results["FFN_accuracy"]
    ffn_nc_accs[j] = nc_results["FFN_accuracy"]
    ffn_n_accs[j] = n_results["FFN_accuracy"]
    ffn_c_accs[j] = c_results["FFN_accuracy"]

    cnn_base_accs[j] = baseline_results["CNN_accuracy"]
    cnn_nc_accs[j] = nc_results["CNN_accuracy"]
    cnn_n_accs[j] = n_results["CNN_accuracy"]
    cnn_c_accs[j] = c_results["CNN_accuracy"]

run 1
Fitting feed forward network with Gaussian(0,0) noise and 0 random contrast
 20/133 [===>..........................] - ETA: 35s - loss: 4.8548 - accuracy: 0.0211

KeyboardInterrupt: 

In [40]:
# save results as pickles 
# ('wb': 'write byte/binary' => write using byte data: in binary mode)
# ffn
fileObj = open('ffn_base_accs.obj', 'wb')
pickle.dump(ffn_base_accs, fileObj)
fileObj.close()

fileObj = open('ffn_nc_accs.obj', 'wb')
pickle.dump(ffn_nc_accs, fileObj)
fileObj.close()

fileObj = open('ffn_n_accs.obj', 'wb')
pickle.dump(ffn_n_accs, fileObj)
fileObj.close()

fileObj = open('ffn_c_accs.obj', 'wb')
pickle.dump(ffn_c_accs, fileObj)
fileObj.close()

# cnn
fileObj = open('cnn_base_accs.obj', 'wb')
pickle.dump(ffn_base_accs, fileObj)
fileObj.close()

fileObj = open('cnn_nc_accs.obj', 'wb')
pickle.dump(ffn_nc_accs, fileObj)
fileObj.close()

fileObj = open('cnn_n_accs.obj', 'wb')
pickle.dump(ffn_n_accs, fileObj)
fileObj.close()

fileObj = open('cnn_c_accs.obj', 'wb')
pickle.dump(ffn_c_accs, fileObj)
fileObj.close()

In [3]:
# deserialize (reload) objects
fileObj = open('./shortsim2/fn_base_accs.obj', 'rb')
ffn_base_accs_reload2 = pickle.load(fileObj)
fileObj.close()
print(ffn_base_accs_reload2)

fileObj = open('./shortsim2/ffn_c_accs.obj', 'rb')
ffn_c_accs_reload2 = pickle.load(fileObj)
fileObj.close()

fileObj = open('./shortsim2/fn_n_accs.obj', 'rb')
ffn_n_accs_reload2 = pickle.load(fileObj)
fileObj.close()

fileObj = open('./shortsim2/ffn_nc_accs.obj', 'rb')
ffn_nc_accs_reload2 = pickle.load(fileObj)
fileObj.close()

fileObj = open('./shortsim2/cnn_base_accs.obj', 'rb')
cnn_base_accs_reload2 = pickle.load(fileObj)
fileObj.close()
print(cnn_base_accs_reload2)

fileObj = open('./shortsim2/cnn_c_accs.obj', 'rb')
cnn_c_accs_reload2 = pickle.load(fileObj)
fileObj.close()

fileObj = open('./shortsim2/cnn_n_accs.obj', 'rb')
cnn_n_accs_reload2 = pickle.load(fileObj)
fileObj.close()

fileObj = open('./shortsim2/cnn_nc_accs.obj', 'rb')
cnn_nc_accs_reload2 = pickle.load(fileObj)
fileObj.close()


print("ANN BASE accuracy:", np.mean(ffn_base_accs_reload2), "with sd:", np.std(ffn_base_accs_reload2))
print("CNN BASE accuracy:", np.mean(cnn_base_accs_reload2), "with sd:", np.std(cnn_base_accs_reload2))
print("ANN NOISY accuracy:", np.mean(ffn_n_accs_reload2), "with sd:", np.std(ffn_n_accs_reload2))
print("CNN NOISY accuracy:", np.mean(cnn_n_accs_reload2), "with sd:", np.std(cnn_n_accs_reload2))
print("ANN GRAY accuracy:", np.mean(ffn_c_accs_reload2), "with sd:", np.std(ffn_c_accs_reload2))
print("CNN GRAY accuracy:", np.mean(cnn_c_accs_reload2), "with sd:", np.std(cnn_c_accs_reload2))
print("ANN NOISY GRAY accuracy:", np.mean(ffn_nc_accs_reload2), "with sd:", np.std(ffn_nc_accs_reload2))
print("CNN NOISY GRAY accuracy:", np.mean(cnn_nc_accs_reload2), "with sd:", np.std(cnn_nc_accs_reload2))

[0.053949903696775436, 0.10019268095493317, 0.060693640261888504, 0.08670520037412643, 0.09248554706573486, 0.08959537744522095, 0.059730250388383865, 0.05202312022447586, 0.06262042373418808, 0.05202312022447586, 0.09152215719223022, 0.05876686051487923, 0.05684007704257965, 0.05202312022447586, 0.08863198757171631, 0.05202312022447586, 0.05202312022447586, 0.09730250388383865, 0.08863198757171631, 0.08863198757171631]
[0.2707129120826721, 0.2678227424621582, 0.2572254240512848, 0.26396918296813965, 0.26396918296813965, 0.2543352544307709, 0.2524084746837616, 0.26204240322113037, 0.24855491518974304, 0.2543352544307709, 0.25626203417778015, 0.25626203417778015, 0.24662813544273376, 0.2447013556957245, 0.25818881392478943, 0.2524084746837616, 0.2475915253162384, 0.24951830506324768, 0.24181117117404938, 0.2774566411972046]
ANN BASE accuracy: 0.07182080931961536 with sd: 0.018280268214057833
CNN BASE accuracy: 0.256310211867094 with sd: 0.008946969233539902
ANN NOISY accuracy: 0.0695086

In [4]:
# deserialize (reload) objects
# original sim: 12 epochs, 20 sims, to gray scale as in to 2d, rather than b&w
fileObj = open('./pickles_shortsim/ffn_base_accs.obj', 'rb')
ffn_base_accs_reload = pickle.load(fileObj)
fileObj.close()
print(ffn_base_accs_reload)

fileObj = open('./pickles_shortsim/ffn_c_accs.obj', 'rb')
ffn_c_accs_reload = pickle.load(fileObj)
fileObj.close()

fileObj = open('./pickles_shortsim/ffn_n_accs.obj', 'rb')
ffn_n_accs_reload = pickle.load(fileObj)
fileObj.close()

fileObj = open('./pickles_shortsim/ffn_nc_accs.obj', 'rb')
ffn_nc_accs_reload = pickle.load(fileObj)
fileObj.close()

fileObj = open('./pickles_shortsim/cnn_base_accs.obj', 'rb')
cnn_base_accs_reload = pickle.load(fileObj)
fileObj.close()
print(cnn_base_accs_reload)

fileObj = open('./pickles_shortsim/cnn_c_accs.obj', 'rb')
cnn_c_accs_reload = pickle.load(fileObj)
fileObj.close()

fileObj = open('./pickles_shortsim/cnn_n_accs.obj', 'rb')
cnn_n_accs_reload = pickle.load(fileObj)
fileObj.close()

fileObj = open('./pickles_shortsim/cnn_nc_accs.obj', 'rb')
cnn_nc_accs_reload = pickle.load(fileObj)
fileObj.close()


print("ANN BASE accuracy:", np.mean(ffn_base_accs_reload), "with sd:", np.std(ffn_base_accs_reload))
print("CNN BASE accuracy:", np.mean(cnn_base_accs_reload), "with sd:", np.std(cnn_base_accs_reload))
print("ANN NOISY accuracy:", np.mean(ffn_n_accs_reload), "with sd:", np.std(ffn_n_accs_reload))
print("CNN NOISY accuracy:", np.mean(cnn_n_accs_reload), "with sd:", np.std(cnn_n_accs_reload))
print("ANN GRAY accuracy:", np.mean(ffn_c_accs_reload), "with sd:", np.std(ffn_c_accs_reload))
print("CNN GRAY accuracy:", np.mean(cnn_c_accs_reload), "with sd:", np.std(cnn_c_accs_reload))
print("ANN NOISY GRAY accuracy:", np.mean(ffn_nc_accs_reload), "with sd:", np.std(ffn_nc_accs_reload))
print("CNN NOISY GRAY accuracy:", np.mean(cnn_nc_accs_reload), "with sd:", np.std(cnn_nc_accs_reload))

[0.05202312022447586, 0.05202312022447586, 0.05202312022447586, 0.09730250388383865, 0.05587668716907501, 0.05202312022447586, 0.05202312022447586, 0.05202312022447586, 0.06551060080528259, 0.08863198757171631, 0.06262042373418808, 0.05202312022447586, 0.09152215719223022, 0.054913293570280075, 0.1011560708284378, 0.05202312022447586, 0.0529865138232708, 0.06262042373418808, 0.08863198757171631, 0.08863198757171631]
[0.2504816949367523, 0.24662813544273376, 0.2697495222091675, 0.2350674420595169, 0.25626203417778015, 0.25144508481025696, 0.2447013556957245, 0.23892100155353546, 0.2658959627151489, 0.25626203417778015, 0.24181117117404938, 0.26204240322113037, 0.26107898354530334, 0.26107898354530334, 0.26685935258865356, 0.2572254240512848, 0.24566474556922913, 0.24662813544273376, 0.24662813544273376, 0.24277456104755402]
ANN BASE accuracy: 0.06632947996258735 with sd: 0.01786376400760015
CNN BASE accuracy: 0.2523603081703186 with sd: 0.009708167213140328
ANN NOISY accuracy: 0.0723025

In [20]:
sim_results_dict = {"Method": ["ANN_BASE","ANN_NOISY","ANN_GRAY", "ANN_NOISY_GRAY","CNN_BASE","CNN_NOISY","CNN_GRAY", "CNN_NOISY_GRAY"],
                    "Mean Accuracy":[np.mean(ffn_base_accs_reload),np.mean(ffn_n_accs_reload),np.mean(ffn_c_accs_reload),np.mean(ffn_nc_accs_reload),
                                     np.mean(cnn_base_accs_reload),np.mean(cnn_n_accs_reload),np.mean(cnn_c_accs_reload),np.mean(cnn_nc_accs_reload)],
                    "SD Accuracy": [np.std(ffn_base_accs_reload),np.std(ffn_n_accs_reload),np.std(ffn_c_accs_reload),np.std(ffn_nc_accs_reload),
                                    np.std(cnn_base_accs_reload),np.std(cnn_n_accs_reload),np.std(cnn_c_accs_reload),np.std(cnn_nc_accs_reload)]}
sim_res_df = pd.DataFrame(sim_results_dict)
sim_res_df

,Method,Mean Accuracy,SD Accuracy
0,ANN_BASE,0.066329,0.017864
1,ANN_NOISY,0.072303,0.017911
2,ANN_GRAY,0.061561,0.013411
3,ANN_NOISY_GRAY,0.059104,0.013122
4,CNN_BASE,0.252360,0.009708
5,CNN_NOISY,0.221484,0.010540
6,CNN_GRAY,0.193256,0.009387
7,CNN_NOISY_GRAY,0.175048,0.006549


In [23]:
# output table to latex
sim_res_df.to_latex

<bound method NDFrame.to_latex of            Method  Mean Accuracy  SD Accuracy
0        ANN_BASE       0.066329     0.017864
1       ANN_NOISY       0.072303     0.017911
2        ANN_GRAY       0.061561     0.013411
3  ANN_NOISY_GRAY       0.059104     0.013122
4        CNN_BASE       0.252360     0.009708
5       CNN_NOISY       0.221484     0.010540
6        CNN_GRAY       0.193256     0.009387
7  CNN_NOISY_GRAY       0.175048     0.006549>

In [5]:
# deserialize (reload) objects
# corrected sim: 12 epochs, 20 sims, fully to gray scale, back as 3d rgb image
fileObj = open('./sim_gray_corrected/ffn_base_accs.obj', 'rb')
ffn_base_accs_reloadc = pickle.load(fileObj)
fileObj.close()
print(ffn_base_accs_reloadc)

fileObj = open('./sim_gray_corrected/ffn_c_accs.obj', 'rb')
ffn_c_accs_reloadc = pickle.load(fileObj)
fileObj.close()

fileObj = open('./sim_gray_corrected/ffn_n_accs.obj', 'rb')
ffn_n_accs_reloadc = pickle.load(fileObj)
fileObj.close()

fileObj = open('./sim_gray_corrected/ffn_nc_accs.obj', 'rb')
ffn_nc_accs_reloadc = pickle.load(fileObj)
fileObj.close()

fileObj = open('./sim_gray_corrected/cnn_base_accs.obj', 'rb')
cnn_base_accs_reloadc = pickle.load(fileObj)
fileObj.close()
print(cnn_base_accs_reloadc)

fileObj = open('./sim_gray_corrected/cnn_c_accs.obj', 'rb')
cnn_c_accs_reloadc = pickle.load(fileObj)
fileObj.close()

fileObj = open('./sim_gray_corrected/cnn_n_accs.obj', 'rb')
cnn_n_accs_reloadc = pickle.load(fileObj)
fileObj.close()

fileObj = open('./sim_gray_corrected/cnn_nc_accs.obj', 'rb')
cnn_nc_accs_reloadc = pickle.load(fileObj)
fileObj.close()


print("ANN BASE accuracy:", np.mean(ffn_base_accs_reloadc), "with sd:", np.std(ffn_base_accs_reloadc))
print("CNN BASE accuracy:", np.mean(cnn_base_accs_reloadc), "with sd:", np.std(cnn_base_accs_reloadc))
print("ANN NOISY accuracy:", np.mean(ffn_n_accs_reloadc), "with sd:", np.std(ffn_n_accs_reloadc))
print("CNN NOISY accuracy:", np.mean(cnn_n_accs_reloadc), "with sd:", np.std(cnn_n_accs_reloadc))
print("ANN GRAY accuracy:", np.mean(ffn_c_accs_reloadc), "with sd:", np.std(ffn_c_accs_reloadc))
print("CNN GRAY accuracy:", np.mean(cnn_c_accs_reloadc), "with sd:", np.std(cnn_c_accs_reloadc))
print("ANN NOISY GRAY accuracy:", np.mean(ffn_nc_accs_reloadc), "with sd:", np.std(ffn_nc_accs_reloadc))
print("CNN NOISY GRAY accuracy:", np.mean(cnn_nc_accs_reloadc), "with sd:", np.std(cnn_nc_accs_reloadc))

[0.08863198757171631, 0.05684007704257965, 0.06551060080528259, 0.05876686051487923, 0.05684007704257965, 0.05876686051487923, 0.06358381360769272, 0.08766859024763107, 0.05202312022447586, 0.05202312022447586, 0.06647399067878723, 0.060693640261888504, 0.08766859024763107, 0.05202312022447586, 0.05202312022447586, 0.08381503075361252, 0.08863198757171631, 0.09826589375734329, 0.08477842062711716, 0.05202312022447586]
[0.2543352544307709, 0.2572254240512848, 0.2350674420595169, 0.25337186455726624, 0.2572254240512848, 0.26204240322113037, 0.25626203417778015, 0.23795761168003082, 0.24277456104755402, 0.23795761168003082, 0.2552986443042755, 0.2524084746837616, 0.2572254240512848, 0.2273603081703186, 0.25337186455726624, 0.23892100155353546, 0.25144508481025696, 0.25818881392478943, 0.2524084746837616, 0.25626203417778015]
ANN BASE accuracy: 0.06835260111838579 with sd: 0.01556845755402284
CNN BASE accuracy: 0.249855487793684 with sd: 0.009301962132233057
ANN NOISY accuracy: 0.065173410

In [16]:
sim_results_dict = {"Method": ["DNN BASE","DNN NOISY","DNN GRAY", "DNN NOISY+GRAY","CNN BASE","CNN NOISY","CNN GRAY", "CNN NOISY+GRAY"],
                    "Mean Accuracy":[np.mean(ffn_base_accs_reloadc),np.mean(ffn_n_accs_reloadc),np.mean(ffn_c_accs_reloadc),np.mean(ffn_nc_accs_reloadc),
                                     np.mean(cnn_base_accs_reloadc),np.mean(cnn_n_accs_reloadc),np.mean(cnn_c_accs_reloadc),np.mean(cnn_nc_accs_reloadc)],
                    "SD": [np.std(ffn_base_accs_reloadc),np.std(ffn_n_accs_reloadc),np.std(ffn_c_accs_reloadc),np.std(ffn_nc_accs_reloadc),
                                    np.std(cnn_base_accs_reloadc),np.std(cnn_n_accs_reloadc),np.std(cnn_c_accs_reloadc),np.std(cnn_nc_accs_reloadc)]}
sim_res_dfc = pd.DataFrame(sim_results_dict)
sim_res_dfc

# output table to latex
print(sim_res_dfc.style.to_latex(position = 't'))

\begin{table}[t]
\begin{tabular}{llrr}
 & Method & Mean Accuracy & SD \\
0 & DNN BASE & 0.068353 & 0.015568 \\
1 & DNN NOISY & 0.065173 & 0.015315 \\
2 & DNN GRAY & 0.062909 & 0.015348 \\
3 & DNN NOISY+GRAY & 0.057033 & 0.009087 \\
4 & CNN BASE & 0.249855 & 0.009302 \\
5 & CNN NOISY & 0.216908 & 0.011855 \\
6 & CNN GRAY & 0.191089 & 0.009587 \\
7 & CNN NOISY+GRAY & 0.177168 & 0.008236 \\
\end{tabular}
\end{table}



In [72]:
# attempt to look at confusion matrix/classification report
# predictions appear to be in shuffled order,
# so performance not matching what is shown when model.evaluate() is run

# test_labels = test_generator.classes_
# pred_digits = np.argmax(model_cnn.predict(test_generator),axis=1)
# from sklearn.metrics import classification_report, confusion_matrix
# confusion_matrix(y_true=test_labels,y_pred=pred_digits)
# print(classification_report(test_labels,pred_digits))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        20
           1       0.00      0.00      0.00        22
           2       0.00      0.00      0.00        22
           3       0.00      0.00      0.00        20
           4       0.00      0.00      0.00        22
           5       0.00      0.00      0.00        24
           6       0.00      0.00      0.00        17
           7       0.00      0.00      0.00        20
           8       0.00      0.00      0.00        21
           9       0.00      0.00      0.00        17
          10       0.08      0.04      0.05        26
          11       0.00      0.00      0.00        20
          12       0.00      0.00      0.00        16
          13       0.00      0.00      0.00        23
          14       0.00      0.00      0.00        16
          15       0.00      0.00      0.00        22
          16       0.00      0.00      0.00        18
          17       0.00    

c:\Users\tyler\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\tyler\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\tyler\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
